In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# Load the Drive helper and mount
# from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')

# load data
# ASD_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ASD_gene_labeled.csv')
# ASD_data.head() 
# fatchd_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FATCHD/FATCHD.csv')
# fatchd_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID_C,BMI01,ETHANL03,CIGT01,MOVE,P_CARB,CHOL,DFIB,P_PROT,P_SFAT,...,ANTA01,ANTA07A,ANTA07B,ECGMA31,HMTA03,APASIU01,APBSIU01,LIPA08,CHMA09,CLASS
0,1,28.767562,0.0,3.0,5.000,55.763035,271.47,15.51,20.085680,9.698284,...,159.0,102.0,112.0,69.0,4.6,1450.0,1700.0,8.0,1.1,0
1,2,34.645233,0.0,3.0,7.250,46.258407,335.55,9.92,18.071302,14.855120,...,164.0,106.0,117.0,73.0,7.6,NaN,NaN,NaN,1.0,0
2,3,28.392191,41.0,2.0,6.250,51.184492,358.20,24.20,22.774525,9.353566,...,164.0,95.0,105.0,73.0,7.4,1500.0,1270.0,12.0,1.1,0
3,4,22.038567,0.0,3.0,5.750,48.461187,220.87,12.62,20.222597,13.905987,...,165.0,73.0,103.0,56.0,5.9,2190.0,770.0,32.0,1.0,0
4,5,30.452441,0.0,2.0,9.125,53.584911,106.73,24.73,16.452840,11.733608,...,163.0,106.0,109.0,63.0,3.7,1310.0,890.0,248.0,1.0,0


In [ ]:
import math
from sklearn.impute import KNNImputer
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
# creating X, Y train for cv and data preprocessing
Y = fatchd_data['CLASS']
X = fatchd_data
X.drop(['ID_C', 'CLASS'], axis=1, inplace=True)
# dealing with na values
# X = X.fillna(0)
imputer = KNNImputer(n_neighbors=2)
X = pd.DataFrame(imputer.fit_transform(X))

In [ ]:
# Adaptive SMOTE
import math
import random
from sklearn.neighbors import NearestNeighbors
from operator import itemgetter, attrgetter
import numpy as np
from numpy import linalg as LA
from sklearn.decomposition import PCA
class Adaptive_Smote:
    def __init__(self,samples, N = 1, k = 5):
        self.n_samples,self.n_attrs=samples.shape
        self.N=N
        self.k=k
        self.samples=samples
        self.newindex=0

    def over_sampling(self):
      N = self.N
      self.synthetic = np.zeros((self.n_samples * N, self.n_attrs))
      neighbors = NearestNeighbors(n_neighbors=self.k).fit(self.samples)
      for i in range(len(self.samples)):
        nnarray = neighbors.kneighbors(self.samples.iloc[[i]].values.reshape((1,-1)),return_distance = False)[0]  #Finds the K-neighbors of a point.
        nnarray = np.delete(nnarray, 0)
        adaptive_nnarray = self.neighborhood_selection(i, nnarray) 
        if adaptive_nnarray != []:
          self._populate(N, i, adaptive_nnarray)
      return self.synthetic

    # neighborhood selection algorithm 
    def neighborhood_selection(self, i, nnarray):
      visible_nnarray = []    	
      for j in range(len(nnarray)):
        ck_nnarray = np.delete(nnarray, j)
        ck = 0
        for z in ck_nnarray:
          inner_pro = np.inner((self.samples.iloc[[i]] - self.samples.iloc[z]), self.samples.iloc[nnarray[j]] - self.samples.iloc[z])
          if inner_pro >= 0:
            ck += 1
          else:
            ck += 0
        if ck == len(ck_nnarray):
          visible_nnarray.append(nnarray[j])   
      return visible_nnarray

      # for each minority class sample i ,choose N of the k nearest neighbors and generate N synthetic samples.
    def _populate(self,N,i,nnarray):
      for j in range(N): 
        nn = random.randint(0,len(nnarray)-1)  #include end
        for attr in range(0, self.n_attrs):
          dif = self.samples.iloc[nnarray[nn]][attr]-self.samples.iloc[[i]][attr]
          gap = random.random()
          self.synthetic[self.newindex][attr] = self.samples.iloc[[i]][attr] + gap*dif
        self.newindex += 1
        # print(j)

In [ ]:
# Inspection Classifiers
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
def inspection_classifier(majority_X, majority_y, min_data, smote_data, threshold):
  # training inspection classifiers
  num_removed_data = 0
  n = math.floor(len(majority_X)/len(min_data))
  split_X = np.array_split(majority_X, n)
  split_y = np.array_split(majority_y, n)
  min_y = pd.Series([1 for i in range(len(min_data))])
  clf_lst = []
  for i in range (0, n):
    df_split_X = pd.DataFrame(split_X[i])
    new_ad_cols = {x: y for x, y in zip(min_data, df_split_X.columns)}
    combined_x = df_split_X.append(min_data.rename(columns=new_ad_cols))
    combined_y = split_y[i].append(min_y)
    rf = RandomForestClassifier(random_state=0)
    rf.fit(combined_x, combined_y)
    clf_lst.append(rf)

  # inspection part
  df_smote = pd.DataFrame(smote_data)
  drop_index = []
  for j in range(0, len(smote_data)): 
    score = 0
    for clf in clf_lst:
      if clf.predict(np.array(df_smote.iloc[j]).reshape(1, -1))[0] == 1:
        score += 1
    # remove
    if score/len(clf_lst) < threshold:
      drop_index.append(j)
      num_removed_data += 1
  df_smote.drop(df_smote.index[drop_index], inplace =True)    
  # print("# of removed data:", num_removed_data)
  return df_smote

In [ ]:
import math
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn import metrics
def score_report_manual(clf, X_test, y_test, y_pred):
  cf_matrix = confusion_matrix(y_test, y_pred)
  tp = cf_matrix[1][1]
  if tp == 0:
    tp += 1
  pr = tp/(tp + cf_matrix[1][0])
  rc = tp/(tp + cf_matrix[0][1])
  f_1 = (2 * pr * rc)/(pr + rc)
  # y_pred_proba = clf.predict_proba(X_test)[:, 1]
  y_pred_proba = (clf.predict_proba(X_test)[:, 1] > 0.5).astype('float')
  precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

  f_score = (2 * precision * recall) / (precision + recall)
  # index = np.argmax(f_score)
  index = np.nanargmax(f_score)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(f_score[index], ndigits = 4)
  fscoreOpt2 = round(f_score[index], ndigits = 4)
  recallOpt = round(recall[index], ndigits = 4)
  precisionOpt = round(precision[index], ndigits = 4)
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))
  print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))
  auc = metrics.auc(recall, precision)
  return precisionOpt, recallOpt, fscoreOpt, auc

In [ ]:
# A SMOTE (proposed method)
# Adaptive SMOTE with inspection

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import statistics
from sklearn.metrics import accuracy_score
random.seed(10)
kf = KFold(n_splits=5, shuffle = True, random_state=0)
for p in [0.75]:
  for K in [[75,20]]:
    for n in [[13,0.5]]:
      acc_lst = []
      f1 = []
      precision_lst = []
      recall_lst = []
      auc_lst = []
      len_df_syn = []
      len_df_maj = []
      print('p:', p,'n:', n, 'k:', K, 'Threshold: 0.5', "ratio: 0.05")
      for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        # Data setting before SMOTE
        X_4 = X.iloc[train_index.tolist()] 
        y_4 = Y.iloc[train_index.tolist()] 
        X_test_4 = X.iloc[test_index.tolist()]
        y_test_4 = Y.iloc[test_index.tolist()]
        train_combined = np.c_[X_4, y_4]
        maj_x = train_combined[train_combined[:,37] == 0]
        maj_x = np.delete(maj_x, 37, 1)
        maj_y = pd.Series([0 for i in range(len(maj_x))])
        df_maj_x1 = pd.DataFrame(maj_x)
        minority_data = train_combined[train_combined[:,37] == 1][0:math.ceil(p*len(train_combined[train_combined[:,37] == 1]))]
        minority_data = np.delete(minority_data, 37, 1)
        df_min_data = pd.DataFrame(minority_data)
        df_maj_x = df_maj_x1
        min_y = pd.Series([1 for i in range(len(df_min_data))])
        if math.ceil(K[1]/100*len(df_min_data)) < 1:
          K_used = K[0]
        else:
          K_used = math.ceil(K[1]/100*len(df_min_data))

        ########################################################## IMPLEMENTATION ##########################################################################
        adaptive_smote = Adaptive_Smote(df_min_data, N = n[0], k = K_used) 
        ad_syn_smote = adaptive_smote.over_sampling()
        inspection_smote_data = inspection_classifier(maj_x, maj_y, df_min_data, ad_syn_smote, 0.75)
        df_syn1 = pd.DataFrame(inspection_smote_data)
        ########################################################## IMPLEMENTATION ##########################################################################

        df_syn = df_syn1.sample(n = math.ceil(n[1]*len(df_maj_x)) - len(df_min_data), replace = False, random_state = 0) ####
        y_adapt_syn = pd.Series([1 for i in range(len(df_syn))])
        new_cols = {x: y for x, y in zip(df_syn, X_4.columns)}
        df_min_x = df_min_data.append(df_syn.rename(columns=new_cols)) ###
        df_min_y = min_y.append(y_adapt_syn) ###

        # add the entire min data with maj data
        new_maj_y = pd.Series([0 for i in range(len(df_maj_x))]) ##########
        X_train4 = df_maj_x.append(df_min_x.rename(columns=new_cols)) #
        y_train4 = new_maj_y.append(df_min_y) ##########
        
        # Training model
        rf_4 = RandomForestClassifier(max_depth = 2, random_state=0)
        rf_4.fit(X_train4, y_train4)
        y_pred_rf_4 = rf_4.predict(X_test_4)
        pr, rc, f_1, auc = score_report_manual(rf_4, X_test_4, y_test_4, y_pred_rf_4)
        f1.append(f_1)
        precision_lst.append(pr)
        recall_lst.append(rc)
        auc_lst.append(auc)
        len_df_syn.append(len(df_syn))
        len_df_maj.append(len(maj_x))
        acc = accuracy_score(y_test_4, y_pred_rf_4)
        acc_lst.append(acc)
        print(f'For fold {fold}:')
        print("Precision" , pr)
        print("Recall" , rc)
        print("F1 Score" ,f_1)
        print("Accuracy", acc)
      print('average_precision', sum(precision_lst)/5)
      print('average_recall', sum(recall_lst)/5)
      print('average_f1', sum(f1)/5)
      print('average_auc', sum(auc_lst)/5)
      print('average_accuracy', sum(acc_lst)/5)
      print('stdev_pr', statistics.stdev(precision_lst))
      print('stdev_rc', statistics.stdev(recall_lst))
      print('stdev_f1', statistics.stdev(f1))
      print('stdev_acc', statistics.stdev(acc_lst))
      print('--------------------------------------------------')
    print('************************************************************')  
  

p: 0.75 n: [13, 0.5] k: [75, 20] Threshold: 0.5 ratio: 0.05
82
Best Threshold: 1.0 with F-Score: 0.2712
Recall: 0.237, Precision: 0.3168
For fold 1:
Precision 0.3168
Recall 0.237
F1 Score 0.2712
Accuracy 0.9201856148491879
81
Best Threshold: 1.0 with F-Score: 0.2097
Recall: 0.2, Precision: 0.2205
For fold 2:
Precision 0.2205
Recall 0.2
F1 Score 0.2097
Accuracy 0.9020881670533643
83
Best Threshold: 1.0 with F-Score: 0.2311
Recall: 0.2283, Precision: 0.2339
For fold 3:
Precision 0.2339
Recall 0.2283
F1 Score 0.2311
Accuracy 0.9104408352668213
83
Best Threshold: 1.0 with F-Score: 0.2682
Recall: 0.28, Precision: 0.2574
For fold 4:
Precision 0.2574
Recall 0.28
F1 Score 0.2682
Accuracy 0.9113689095127611
80
Best Threshold: 1.0 with F-Score: 0.2662
Recall: 0.2333, Precision: 0.3097
For fold 5:
Precision 0.3097
Recall 0.2333
F1 Score 0.2662
Accuracy 0.9103992571959145
average_precision 0.26765999999999995
average_recall 0.23572
average_f1 0.24928
average_auc 0.27573753775658094
average_accurac